In [ ]:
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()


In [ ]:
dataset = Dataset.get_by_name(ws, "student-data-final")
df = dataset.to_pandas_dataframe()


In [ ]:
X = df.drop(columns=['G3'])
y = df['G3']


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('num', 'passthrough', num_cols)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.pipeline import Pipeline

rf = RandomForestRegressor(n_estimators=100, random_state=42)
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)

voting_model = Pipeline([
    ('preprocess', preprocessor),
    ('ensemble', VotingRegressor([
        ('rf', rf),
        ('gb', gb)
    ]))
])

voting_model.fit(X_train, y_train)


In [ ]:
import json
import joblib

expected_columns = X.columns.tolist()

with open("input_schema.json", "w") as f:
    json.dump(expected_columns, f)

sample_input = X.iloc[0].to_dict()
with open("sample_input.json", "w") as f:
    json.dump(sample_input, f)

joblib.dump(voting_model, "student_g3_model.pkl")


In [ ]:
from azureml.core import Workspace, Model
import joblib

ws = Workspace.from_config()

model = Model.register(
    workspace=ws,
    model_path="student_g3_model.pkl",   
    model_name="votingensemble_student_final",
    description="Voting ensemble model predicting G3",
    tags={"type": "votingensemble"}
)

print(f"Model registered: {model.name}, version: {model.version}")


In [ ]:
from azureml.core import Workspace, Model, Environment
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig


In [ ]:
ws = Workspace.from_config()
print("Workspace:", ws.name)
print("Subscription:", ws.subscription_id)
print("Resource group:", ws.resource_group)


In [ ]:
model = Model(ws, name="votingensemble_student_final")
print(f"Model name: {model.name}, version: {model.version}")


In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(
    name="student_g3_env",
    file_path="environment.yml"
)


In [ ]:
inference_config = InferenceConfig(
    entry_script="score.py",  
    environment=env           
)


In [ ]:
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    auth_enabled=True,   
    description="Voting ensemble model predicting student final grade"
)


In [ ]:
service_name = "student-g3-service"

service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True  
)

service.wait_for_deployment(show_output=True)
print(f"Service state: {service.state}")
print(f"Scoring URI: {service.scoring_uri}")
if service.auth_enabled:
    print(f"Primary key: {service.get_keys()[0]}")
